In [1]:
import pandas as pd


def clean_name(first_name, last_name):
    import unicodedata

    if not isinstance(first_name, str):
        first_name = str(first_name) if first_name is not None else ""
    if not isinstance(last_name, str):
        last_name = str(last_name) if last_name is not None else ""

    first_name = first_name.lower().strip()
    last_name = last_name.lower().strip()

    def normalize_chars(text):
        text = unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode('ASCII')
        return text

    first_name = normalize_chars(first_name)
    last_name = normalize_chars(last_name)

    for char in ['-', "'", "`", ".", ",", "&", "'"]:  # Added apostrophe variants
        first_name = first_name.replace(char, ' ')
        last_name = last_name.replace(char, ' ')

    while '  ' in first_name:
        first_name = first_name.replace('  ', ' ')
    while '  ' in last_name:
        last_name = last_name.replace('  ', ' ')

    first_name = ' '.join(word.capitalize() for word in first_name.split())
    last_name = ' '.join(word.capitalize() for word in last_name.split())

    full_name = f"{first_name} {last_name}".strip()

    return full_name


def clean_text(text):
    if not isinstance(text, str):
        return text

    text = text.lower()

    for char in ['&', ',', '-']:
        text = text.replace(char, ' ')

    text = text.replace(' and ', ' ')

    while '  ' in text:
        text = text.replace('  ', ' ')

    return text.strip()


def process_ep_voting_data(rcv_files, voted_docs_files):
    if len(rcv_files) != len(voted_docs_files):
        raise ValueError("The lists of RCV files and Voted docs files must have the same length")

    all_data = []

    for i, (rcv_file, voted_doc_file) in enumerate(zip(rcv_files, voted_docs_files)):
        print(f"Processing files {i + 1}/{len(rcv_files)}: {rcv_file} and {voted_doc_file}")

        if "EP6" in rcv_file:
            ep_session = "EP6"
            vote_start_index = 10
            rcv_data = pd.read_excel(rcv_file, header=1)
        elif "EP7" in rcv_file:
            ep_session = "EP7"
            vote_start_index = 9
            rcv_data = pd.read_excel(rcv_file, sheet_name=0)
        elif "EP8" in rcv_file:
            ep_session = "EP8"
            vote_start_index = 9
            rcv_data = pd.read_excel(rcv_file, sheet_name=0)
        elif "EP9" in rcv_file:
            ep_session = "EP9"
            vote_start_index = 10
            rcv_data = pd.read_excel(rcv_file, sheet_name=0)
        else:
            ep_session = "Unknown"
            rcv_data = pd.read_excel(rcv_file, sheet_name=0)
            print("UNKNOWN SESSION")

        rcv_data = rcv_data.dropna(how='all')

        voted_docs = pd.read_excel(voted_doc_file)

        # Get vote columns headers (index)
        vote_columns = rcv_data.columns[vote_start_index:].tolist()

        votes_df = process_votes_ep(rcv_data, voted_docs, vote_columns, ep_session=ep_session)

        print(f"Should be total length: {len(rcv_data) * len(voted_docs)}")
        print(f"Got length: {len(votes_df)}")

        # Add EP session information
        votes_df['ep_session'] = ep_session

        # Append to the list of results
        all_data.append(votes_df)

    # Concatenate all dataframes
    combined_df = pd.concat(all_data, ignore_index=True)

    # Perform final cleaning
    combined_df = clean_combined_data(combined_df)

    return combined_df


def process_votes_ep(rcv_data, voted_docs, vote_columns, ep_session=None):
    """Process voting data for EP7, EP8, EP9 sessions"""

    total_skipped = 0

    if ep_session == 'EP6':
        date = 'date'
        title = 'title'
        policy_area = 'main_policy_name'
        vote_id_key = 'euro_act_id'
        author = 'author_name'

        mep_id_key = 'WebisteEpID'

    else:
        date = 'Date'
        title = 'Title'
        policy_area = 'De'
        vote_id_key = 'Vote ID'
        author = 'Author'

        mep_id_key = 'WebisteEpID'

        if ep_session == 'EP7':
            mep_id_key = 'MEP ID'

        if ep_session == 'EP8':
            policy_area = "De/Policy area"

        elif ep_session == 'EP9':
            policy_area = 'Policy area'

    # Create a dictionary to map vote IDs to vote information
    vote_info = {}
    for _, row in voted_docs.iterrows():
        vote_info[str(row[vote_id_key])] = {
            'date': row[date],
            'title': row[title],
            'policy_area': row[policy_area],
            'author': author,
        }

    # Create a list to store results
    results = []

    # Process each MEP's votes
    for _, mep_row in rcv_data.iterrows():
        country = mep_row['Country']
        party = mep_row['Party']
        epg = mep_row['EPG']

        first_name = mep_row['Fname']
        last_name = mep_row['Lname']

        mep_id = mep_row[mep_id_key]

        # Process each vote for this MEP
        for vote_col in vote_columns:

            vote_col = str(vote_col)
            vote_code = f'{ep_session}-{vote_col}'

            if vote_col not in vote_info:
                total_skipped += 1
                continue

            try:
                mep_vote = mep_row[str(vote_col)]
            except Exception as e:
                mep_vote = mep_row[int(vote_col)]

            if mep_vote == 0:
                continue

            info = vote_info[vote_col]

            results.append({
                'full name': clean_name(first_name, last_name),
                'country': country,
                'national_party': party,
                'epg': epg,
                'mep_id': mep_id,
                'vote_code': vote_code,
                'vote': mep_vote,
                'date': info['date'],
                'title': info['title'],
                'policy_area': clean_text(info['policy_area']),
            })

    print(f"Were not able to match: {total_skipped} votes")
    return pd.DataFrame(results)


def clean_combined_data(df):
    df['date'] = pd.to_datetime(df['date'], errors='coerce')
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['policy_area_cleaned'] = df['policy_area'].str.strip().str.lower()

    return df


voted_docs_files = ["VoteWatch-EP-voting-data_2004-2022/EP6_Voted docs.xlsx",
                    "VoteWatch-EP-voting-data_2004-2022/EP7_Voted docs.xlsx",
                    "VoteWatch-EP-voting-data_2004-2022/EP8_Voted docs.xlsx",
                    "VoteWatch-EP-voting-data_2004-2022/EP9_Voted docs.xlsx"]
rcv_files = ["VoteWatch-EP-voting-data_2004-2022/EP6_RCVs_2022_06_13.xlsx",
             "VoteWatch-EP-voting-data_2004-2022/EP7_RCVs_2014_06_19.xlsx",
             "VoteWatch-EP-voting-data_2004-2022/EP8_RCVs_2019_06_25.xlsx",
             "VoteWatch-EP-voting-data_2004-2022/EP9_RCVs_2022_06_22.xlsx"]

combined_df = process_ep_voting_data(rcv_files, voted_docs_files)

# Save the combined dataframe
output_file = "ep_voting_data_combined.csv"
combined_df.to_csv(output_file, index=False)

Processing files 1/4: VoteWatch-EP-voting-data_2004-2022/EP6_RCVs_2022_06_13.xlsx and VoteWatch-EP-voting-data_2004-2022/EP6_Voted docs.xlsx
Were not able to match: 0 votes
Should be total length: 5827060
Got length: 4759840
Processing files 2/4: VoteWatch-EP-voting-data_2004-2022/EP7_RCVs_2014_06_19.xlsx and VoteWatch-EP-voting-data_2004-2022/EP7_Voted docs.xlsx
Were not able to match: 0 votes
Should be total length: 5937733
Got length: 5233859
Processing files 3/4: VoteWatch-EP-voting-data_2004-2022/EP8_RCVs_2019_06_25.xlsx and VoteWatch-EP-voting-data_2004-2022/EP8_Voted docs.xlsx
Were not able to match: 0 votes
Should be total length: 8796216
Got length: 7696506
Processing files 4/4: VoteWatch-EP-voting-data_2004-2022/EP9_RCVs_2022_06_22.xlsx and VoteWatch-EP-voting-data_2004-2022/EP9_Voted docs.xlsx


/Users/mathiaskroismoller/anaconda3/envs/ada/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


Were not able to match: 0 votes
Should be total length: 10915249
Got length: 9520348


In [2]:

# Get all column headers as a list
headers_list = combined_df.columns.tolist()

# Print the list of headers
print("Column headers:")
print(headers_list)

# Print the total number of headers
print(f"\nTotal number of headers: {len(headers_list)}")

Column headers:
['full name', 'country', 'national_party', 'epg', 'mep_id', 'vote_code', 'vote', 'date', 'title', 'policy_area', 'ep_session', 'year', 'month', 'policy_area_cleaned']

Total number of headers: 14


In [26]:

import pandas as pd

df = pd.read_csv('ep_voting_data_combined.csv')

headers_list = list(df.columns)
print(headers_list)
print(df['vote'].unique())

['full name', 'country', 'national_party', 'epg', 'mep_id', 'vote_code', 'vote', 'date', 'title', 'policy_area', 'ep_session', 'year', 'month', 'policy_area_cleaned']
[1. 5. 2. 3. 4. 6.]


In [27]:
df.head()

,full name,country,national_party,epg,mep_id,vote_code,vote,date,title,policy_area,ep_session,year,month,policy_area_cleaned
0,Adamos Adamou,Cyprus,Anorthotiko Komma Ergazomenou Laou - Aristera ...,Confederal Group of the European United Left -...,28469.0,EP6-1,1.0,2004-09-15,EU-Mauritius fishing agreement,fisheries,EP6,2004.0,9.0,fisheries
1,Adamos Adamou,Cyprus,Anorthotiko Komma Ergazomenou Laou - Aristera ...,Confederal Group of the European United Left -...,28469.0,EP6-2,1.0,2004-09-15,EU-Mauritius fishing agreement,fisheries,EP6,2004.0,9.0,fisheries
2,Adamos Adamou,Cyprus,Anorthotiko Komma Ergazomenou Laou - Aristera ...,Confederal Group of the European United Left -...,28469.0,EP6-3,1.0,2004-09-15,EU-Madagascar Fishing agreement,fisheries,EP6,2004.0,9.0,fisheries
3,Adamos Adamou,Cyprus,Anorthotiko Komma Ergazomenou Laou - Aristera ...,Confederal Group of the European United Left -...,28469.0,EP6-4,1.0,2004-09-15,EU-Madagascar Fishing agreement,fisheries,EP6,2004.0,9.0,fisheries
4,Adamos Adamou,Cyprus,Anorthotiko Komma Ergazomenou Laou - Aristera ...,Confederal Group of the European United Left -...,28469.0,EP6-5,1.0,2004-09-15,EU-Cape Verde Fishing agreement,fisheries,EP6,2004.0,9.0,fisheries


## Cleaning the group names

In [28]:
# 1) Define a mapping from full names → abbreviations
epg_map = {
    'Confederal Group of the European United Left - Nordic Green Left': 'The Left',
    'Independence/Democracy Group':                           'IDG',
    "Group of the European People's Party (Christian Democrats) and European Democrats": 'EPP',
    "Group of the European People's Party (Christian Democrats)":                     'EPP',
    'Non-attached Members':                                     'NI',
    'Group of the Alliance of Liberals and Democrats for Europe': 'ALDE',
    'Socialist Group in the European Parliament':                'S&D',
    'Group of the Progressive Alliance of Socialists and Democrats in the European Parliament': 'S&D',
    'Union for Europe of the Nations Group':                     'UEN',
    'Group of the Greens/European Free Alliance':                'Greens/EFA',
    'Europe of freedom and democracy Group':                     'EFD',
    'Europe of Freedom and Direct Democracy Group':              'EFDD',
    'Europe of Nations and Freedom Group':                       'ENF',
    'European Conservatives and Reformists Group':               'ECR',
}

# 2) Replace (and leave existing abbreviations intact)
df['epg'] = df['epg'].replace(epg_map)

# 4) Check
print(df['epg'].unique())


['The Left' 'IDG' 'EPP' 'NI' 'ALDE' 'S&D' 'UEN' 'Greens/EFA' 'EFD' 'ECR'
 nan 'EFDD' 'ENF' 'REG']


## Rice Index

In [29]:
def rice_index(yes, no, abstain):
    return abs(yes - no) / (yes + no + abstain)

## Overall parliament agreement by year

In [30]:
# keep only real votes
df_votes = df[df['vote_code'].notna() & df['vote'].isin([1,2,3])].copy()

# count yes/no/abstain per roll-call (vote_code) & year
counts = (
    df_votes
    .groupby(['vote_code','year'])['vote']
    .value_counts()
    .unstack(fill_value=0)
    .rename(columns={1:'yes', 2:'no', 3:'abstain'})
    .reset_index()
)

# compute Rice for each vote_code
counts['rice'] = counts.apply(lambda r: rice_index(r.yes, r.no, r.abstain), axis=1)

# average across all vote_codes in each year
yearly = (
    counts
    .groupby('year')['rice']
    .mean()
    .reset_index()
    .sort_values('year')
)

In [32]:
# 5. Within-party agreement by year
#    Count yes/no/abstain per (party, vote_code, year)
pv = (
    df_votes
    .groupby(['epg', 'vote_code', 'year'])['vote']
    .value_counts()
    .unstack(fill_value=0)
    .rename(columns={1: 'yes', 2: 'no', 3: 'abstain'})
    .reset_index()
)
pv['rice'] = pv.apply(lambda r: rice_index(r.yes, r.no, r.abstain), axis=1)

# Average Rice per party & year, pivot so each party is a column
party_year = (
    pv
    .groupby(['epg', 'year'])['rice']
    .mean()
    .reset_index()
    .pivot(index='year', columns='epg', values='rice')
    .sort_index()
)


In [39]:
# 6. Agreement by policy area by year
pp = (
    df_votes
    .groupby(['policy_area_cleaned', 'vote_code', 'year'])['vote']
    .value_counts()
    .unstack(fill_value=0)
    .rename(columns={1: 'yes', 2: 'no', 3: 'abstain'})
    .reset_index()
)
pp['rice'] = pp.apply(lambda r: rice_index(r.yes, r.no, r.abstain), axis=1)

policy_year = (
    pp
    .groupby(['policy_area_cleaned', 'year'])['rice']
    .mean()
    .reset_index()
    .pivot(index='year', columns='policy_area_cleaned', values='rice')
    .sort_index()
)


KeyError: 'epg'

In [37]:
import pandas as pd
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, CheckboxGroup, CustomJS
from bokeh.layouts import column

# 7. PLOTS with Bokeh
output_notebook()

# 7.1 Parliament-wide static line
p1 = figure(title="Parliament-wide Agreement (Rice Index)",
            x_axis_label='Year', y_axis_label='Rice index',
            width=700, height=350)
p1.line(yearly['year'], yearly['rice'], line_width=2)
show(p1)

# 7.2 Interactive: Agreement by Party
source_party = ColumnDataSource(
    data=dict(year=party_year.index, **{c: party_year[c].values for c in party_year.columns})
)
p2 = figure(title="Agreement by Party",
            x_axis_label='Year', y_axis_label='Rice',
            width=700, height=350)

lines = {}
for party in party_year.columns:
    lines[party] = p2.line('year', party, source=source_party, line_width=2, visible=True)

checkbox_parties = CheckboxGroup(labels=list(party_year.columns),
                                 active=list(range(len(party_year.columns))))

callback_party = CustomJS(args=dict(lines=lines, labels=checkbox_parties.labels), code="""
    const active = cb_obj.active.map(i => labels[i]);
    for (let p of labels) {
        lines[p].visible = active.includes(p);
    }
""")
checkbox_parties.js_on_change('active', callback_party)

show(column(checkbox_parties, p2))

# 7.3 Interactive: Agreement by Policy Area
source_policy = ColumnDataSource(
    data=dict(year=policy_year.index, **{c: policy_year[c].values for c in policy_year.columns})
)
p3 = figure(title="Agreement by Policy Area",
            x_axis_label='Year', y_axis_label='Rice',
            width=700, height=350)

pol_lines = {}
for area in policy_year.columns:
    pol_lines[area] = p3.line('year', area, source=source_policy, line_width=2, visible=True)

checkbox_policies = CheckboxGroup(labels=list(policy_year.columns),
                                 active=list(range(len(policy_year.columns))))

callback_pol = CustomJS(args=dict(lines=pol_lines, labels=checkbox_policies.labels), code="""
    const active = cb_obj.active.map(i => labels[i]);
    for (let area of labels) {
        lines[area].visible = active.includes(area);
    }
""")
checkbox_policies.js_on_change('active', callback_pol)

show(column(checkbox_policies, p3))

Loading BokehJS ...

In [35]:
print(df['epg'].unique())

['The Left' 'IDG' 'EPP' 'NI' 'ALDE' 'S&D' 'UEN' 'Greens/EFA' 'EFD' 'ECR'
 nan 'EFDD' 'ENF' 'REG']


In [40]:
# 1️⃣ Count yes/no/abstain per (party, policy_area)
pp = (
    df_votes
    .groupby(['epg','policy_area_cleaned'])['vote']
    .value_counts()
    .unstack(fill_value=0)
    .rename(columns={1:'yes',2:'no',3:'abstain'})
    .reset_index()
)

# 2️⃣ Rice index
pp['rice'] = pp.apply(lambda r: rice_index(r.yes, r.no, r.abstain), axis=1)

# 3️⃣ Pivot so each party is a column, rows are policy_area
party_policy = (
    pp
    .pivot(index='policy_area_cleaned', columns='epg', values='rice')
    .fillna(0)               # in case some party never voted in an area
    .sort_index()
)

In [53]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, Select, CustomJS
from bokeh.layouts import column

output_notebook()

# 1️⃣ Prepare the CDS with one extra “y” field that we’ll reassign
policies = list(party_policy.index)
parties  = list(party_policy.columns)

data = { 'policy_area': policies }
for p in parties:
    data[p] = party_policy[p].values.tolist()

# initialize “y” to the first party
initial = parties[0]
data['y'] = data[initial]

source = ColumnDataSource(data)

# 2️⃣ Make the figure (categorical x-axis)
p = figure(x_range=policies,
           title="Rice‐score by Policy Area",
           x_axis_label='Policy Area',
           y_axis_label='Rice index',
           width=800, height=800)
p.vbar(x='policy_area', top='y', source=source, width=0.8)
p.xaxis.major_label_orientation = 1.2  # rotate labels so they don’t overlap

# 3️⃣ Dropdown widget
dropdown = Select(title="Party:", value=initial, options=parties)

# 4️⃣ JS callback to swap in the chosen party’s column
callback = CustomJS(args=dict(src=source), code="""
    const d = src.data;
    const choice = cb_obj.value;
    d['y'] = d[choice];
    src.change.emit();
""")
dropdown.js_on_change('value', callback)

# 5️⃣ Layout
show(column(dropdown, p))


Loading BokehJS ...